This notebook aims estimating the structure factor and the pair correlation function of the Pushed Poisson point process

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 12
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=12)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
from mcrppy.spatial_windows import BoxWindow, BallWindow
from mcrppy.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow as BallWindow_sf
from structure_factor.spatial_windows import BoxWindow as BoxWindow_sf
from structure_factor.point_pattern import PointPattern as PointPattern_sf

def transform_to_structure_factor_object(point_pattern):
    if isinstance(point_pattern.window, BallWindow):
        window = BallWindow_sf(center=point_pattern.window.center,
                    radius=point_pattern.window.radius)
    else:
        print( type(point_pattern.window))
        window = BoxWindow_sf(bounds=point_pattern.window.bounds)
    sf_object= PointPattern_sf(
        points=point_pattern.points, 
        window=window,
        intensity=point_pattern.intensity)
    return sf_object

In [ ]:
from mcrppy.point_processes import HomogeneousPoissonPointProcess
import math
import numpy as np

# 1- d=2

In [ ]:
intensity = 1/math.pi
poisson = HomogeneousPoissonPointProcess(intensity)

d=2
r=150
window = BallWindow(center=[0]*d, radius=r)
_, repelled_pp = poisson.generate_repelled_point_pattern(window, seed=1, nb_cores=4)
print("Nb points =", repelled_pp.points.shape[0])

In [ ]:
repelled_pp_sf_ball = transform_to_structure_factor_object(repelled_pp)
l = 2*r/math.sqrt(d)
window_2 = BoxWindow_sf(bounds=[[-l/2,l/2]]*d)
repelled_pp_sf_box = repelled_pp_sf_ball.restrict_to_window(window_2)
# fig, ax = plt.subplots(1,2, figsize=(6,3))
# repelled_pp_sf_ball.plot(axis=ax[0])
# repelled_pp_sf_box.plot(axis=ax[1])
# plt.show()

## 1.1 Structure Factor

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_isotropic = StructureFactor(repelled_pp_sf_ball)
sf = StructureFactor(repelled_pp_sf_box)

In [ ]:
# scattering intensity
k, si = sf.scattering_intensity(k_max=8, meshgrid_shape=(150,150))

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
sf.plot_non_isotropic_estimator(
    k,
    si,
    axes=ax,
    plot_type="radial",
    error_bar=True,
    bins=80,
    scale="log",
    label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$",
    file_name="sf_si_repelled_poisson.pdf"
)

plt.tight_layout(pad=1)

In [ ]:
# Bartlett's isotropic estimator on allowed wavenumbers
# Running time = 156 s
import time

start_time = time.time()
k_norm, s_bi = sf_isotropic.bartlett_isotropic_estimator(nb_values=250)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#Plot
fig, axis = plt.subplots(figsize=(4,3))
sf_isotropic.plot_isotropic_estimator(k_norm, s_bi, 
                                      axis=axis,
                                    label=r"$\widehat{S}_{\mathrm{BI}}(k)$")

## 1.2 Pair correlation function

In [ ]:
# pcf.fv
import structure_factor.pair_correlation_function as pcf
from mcrppy.repelled_point_process import epsilon_critical

r_max=3
pcf_fv_2d = pcf.estimate(repelled_pp_sf_ball, 
                         method="fv", 
                         Kest=dict(rmax=r_max), 
                         fv=dict(method="b", spar=0.2)
                        )

In [ ]:
epsilon = epsilon_critical(d, intensity)
# Plot
pcf_fv_values = pcf_fv_2d["pcf"]
r_values = pcf_fv_2d["r"]
fig, axis = plt.subplots(figsize=(5,4))
axis.plot(r_values, pcf_fv_values, c="grey", marker=".", label="fv", markersize=4)
axis.vlines(epsilon, ymin=np.nanmin(pcf_fv_values), ymax=np.nanmax(pcf_fv_values), 
             colors="g", linestyles='dashed', label=r"$\varepsilon_0$")
axis.hlines(1,xmax=max(r_values), xmin=min(r_values), colors="k", linestyles='dashed', label="Poisson")
axis.legend()
plt.tight_layout()
plt.show()

In [ ]:
# pcf.ppp
r= np.linspace(0, r_max, 100)
pcf_ppp_2d = pcf.estimate(repelled_pp_sf_ball, 
                         method="ppp", 
                         r=r)

In [ ]:
epsilon = epsilon_critical(d, intensity)
# Plot
pcf_ppp_trans = pcf_ppp_2d["trans"]
pcf_ppp_iso = pcf_ppp_2d["iso"]
r_values_2 = pcf_ppp_2d["r"]
fig, axis = plt.subplots(figsize=(5,4))

axis.plot(r_values_2, pcf_ppp_trans, c="b", marker="*", label="ppp", markersize=4)
axis.vlines(epsilon, ymin=np.nanmin(pcf_ppp_trans), 
            ymax=np.nanmax(pcf_ppp_trans), 
             colors="g", linestyles='dashed', label=r"$\varepsilon_0$")
axis.hlines(1,xmax=max(r_values_2), xmin=min(r_values_2), colors="k", linestyles='dashed', label="Poisson")
axis.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.rc('axes', labelsize=12)

epsilon = epsilon_critical(d, intensity)
fig, ax = plt.subplots(1, 2, figsize=(9,3))

# sf.plot_non_isotropic_estimator(k, si, axes=ax[0],
#     plot_type="radial", error_bar=True, bins=80, scale="log",
#     label=r"$\widehat{S}_{\mathrm{SI}}(\mathbf{k})$"
# )
# ax[0].set_xlabel(r"$\|\mathbf{k}\|_2$")
# ax[0].set_ylabel(r"$S(\mathbf{k})$")

ax[0].scatter(k_norm, s_bi, color="grey", s=4, label=r"$\widehat{S}_{\mathrm{BI}}(k)$")
ax[0].hlines(1, xmax=max(k_norm), xmin=min(k_norm), colors="k", linestyles='dashed', label="Poisson")
ax[0].legend()
ax[0].set_xlabel(r"$k$")
ax[0].set_ylabel(r"$S(k)$")

ax[1].plot(r_values_2, pcf_ppp_trans, c="b", marker="*", label="ppp", markersize=4)
ax[1].plot(r_values, pcf_fv_values, c="grey", marker=".", label="fv", markersize=4)
ax[1].hlines(1, xmax=max(r_values), xmin=min(r_values), colors="k", linestyles='dashed', label="Poisson")
ax[1].set_xlabel(r"$r$")
ax[1].set_ylabel(r"$g(r)$")
ax[1].legend()
plt.tight_layout()
plt.savefig("pcf_and_sf_2D.pdf")
plt.show()
